In [69]:
import requests

def crawl(keyword):
    url = "https://search.shopping.naver.com/search/category.nhn?pagingIndex=1&pagingSize=40&viewType=list&sort=review&cat_id=50000202"
    data = requests.get(url)
    print(data.status_code, url)
    return data.content

In [70]:
from bs4 import BeautifulSoup

def getProductInfo(li):
    # print(li)
    img = li.find("img")
    alt = img['alt']
    priceReload = li.find("span", {"class":"_price_reload"})
    date = li.find("span", {"class":"date"})
    categ = li.find('span', {'class':'depth'})
    categ2 = categ.find_all('a')[2]
    reviewnum = li.find('span',{'class':'etc'})
    reviewnum2 = reviewnum.find('a em')
#     reviewnum3 = reviewnum2.find('em')
    aTit = li.find("a", {"class":"tit"})
    href = aTit['href']
    r = requests.get(href)

    return {"name":alt, "price":priceReload.text.replace(",", ""), "link":r.url[:250], "date":date.text.replace(",", "")[4:], 'categ':categ2.text}

def parse(pageString):
    bsObj = BeautifulSoup(pageString, "html.parser")
    ul = bsObj.find("ul", {"class":"goods_list"})
    lis = ul.findAll("li", {"class":"_itemSection"})

    products = []
    for li in lis:
        try:
            product = getProductInfo(li)
            products.append(product)
        except:
            print("--error--")

    return products

In [71]:
# import requests
# from libs.naver_shopping2.parser import parse
import json

def crawl(pageNo):
    url = "https://search.shopping.naver.com/search/category.nhn?pagingIndex={}&pagingSize=40&viewType=list&sort=review&cat_id=50000202".format(pageNo)
    data = requests.get(url)
    print(data, url)
    return data.content

totalProducts = []
for pageNo in range(1, 10+1):
    pageString = crawl(pageNo)
    products = parse(pageString)
    totalProducts += products

print(totalProducts)
print(len(totalProducts))


file = open("men.json", "w+")
file.write(json.dumps(totalProducts))

<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=1&pagingSize=40&viewType=list&sort=review&cat_id=50000202
<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=2&pagingSize=40&viewType=list&sort=review&cat_id=50000202
--error--
<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=3&pagingSize=40&viewType=list&sort=review&cat_id=50000202
--error--
<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=4&pagingSize=40&viewType=list&sort=review&cat_id=50000202
<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=5&pagingSize=40&viewType=list&sort=review&cat_id=50000202
<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=6&pagingSize=40&viewType=list&sort=review&cat_id=50000202
--error--
--error--
<Response [200]> https://search.shopping.naver.com/search/category.nhn?pagingIndex=7&pagingSize=40&viewType=list&s

172188

In [73]:
import pandas as pd

df = pd.read_json("men.json")

print(df.count())

writer = pd.ExcelWriter("men6.xlsx")
df.to_excel(writer, "sheet1")
writer.save()
#openpyxl

categ    393
date     393
link     393
name     393
price    393
dtype: int64
